Decoding molecular and cellular heterogeneity of mouse nucleus accumbens

ShortName: chen2021decoding

Steps of processing the data from raw to Anndata:

In [1]:
# 1, Download the raw data from https://download.brainimagelibrary.org/fc/4c/fc4c2570c3711952/

In [1]:
from soview import *
import shutil
import os
import gc
import pandas as pd

scanpy==1.8.2 anndata==0.8.0 umap==0.5.2 numpy==1.20.3 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.1 python-igraph==0.9.8 pynndescent==0.5.5
squidpy==1.1.2


In [2]:
raw_path = '/aaa/jianhuayao/project2/zhiyuanyuan/PUBDT/st/merfish/chen2021decoding/raw'
h5ad_path = '/aaa/jianhuayao/project2/zhiyuanyuan/PUBDT/st/merfish/chen2021decoding/h5ad'

In [3]:
adata = sc.read_h5ad(f'{raw_path}/counts.h5ad')

AnnData object with n_obs × n_vars = 720037 × 253

In [4]:
segment_matrix_raw = pd.read_csv(f'{raw_path}/segmented_cells_mouse1sample1.csv',dtype='str',delimiter=',',skiprows=1,names=['cell_id','x','y','slide_id'],quotechar='\"')



In [5]:
cell_id_list = np.array(segment_matrix_raw['cell_id'])

In [6]:
x_list = []
y_list = []
for i in range((segment_matrix_raw.shape[0])):
    x_boundary = np.array(segment_matrix_raw['x'][i].split(', ')).astype('float')
    y_boundary = np.array(segment_matrix_raw['y'][i].split(', ')).astype('float')
    cur_x = np.mean(x_boundary)
    cur_y = np.mean(y_boundary)
    x_list.append(cur_x)
    y_list.append(cur_y)

# bulk run

In [7]:
segment_matrix_raw

,cell_id,x,y,slide_id
95134815444313284430238430973583266709,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-1898.6989945679902, -1898.6989945679902, -...","[[[-6762.291238337755, -6762.400238335133, -67...",mouse1_slice10
146727839676135625910533812192046824104,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-7690.556640625, -7690.45854062736, -7690.4...","[[[349.8025565832853, 349.900656580925, 349.91...",mouse1_slice1
38239271305848644177349625612546831841,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-5109.714744976163, -5109.714744976163, -51...","[[[-13.366344302892685, -13.47534430027008, -1...",mouse1_slice1
273909524828856550796800662890663820367,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[], [[-548.4447690755129, -548.4556690752506,...","[[], [[-1343.756284326315, -1343.7671843260528...",mouse1_slice7
38503446544364653836530109500533049442,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-2314.0678721278905, -2314.0678721278905, -...","[[[-7926.648239195347, -7926.757239192724, -79...",mouse1_slice10
...,...,...,...,...
190532580602847208765426042784067116922,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-6080.298815688491, -6080.298815688491, -60...","[[[-5843.707589745522, -5843.816589742899, -58...",mouse1_slice4
77882973113356558386650710885612647206,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-2165.951870879531, -2165.9627708792686, -2...","[[[-7942.889238804579, -7942.900138804316, -79...",mouse1_slice10
278403952600545445484867687324605440396,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-6528.461819341779, -6528.461819341779, -65...","[[[-6495.591588497162, -6495.700588494539, -64...",mouse1_slice4
31216892709811787045685660252250611163,"[0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]","[[[-952.5449932724238, -952.5449932724238, -95...","[[[-6828.999236732721, -6829.108236730099, -68...",mouse1_slice10


In [6]:
segment_matrix_raw.iloc[i]

cell_id                   [0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0]
x           [[[-1898.6989945679902, -1898.6989945679902, -...
y           [[[-6762.291238337755, -6762.400238335133, -67...
slide_id                                       mouse1_slice10
Name: 95134815444313284430238430973583266709, dtype: object

In [13]:
l = eval(segment_matrix_raw['x'][0])
np.mean(np.hstack(l))

[[[-1898.6989945679902,
   -1898.6989945679902,
   -1898.6989945679902,
   -1898.709894567728,
   -1898.8079945653676,
   -1898.8079945653676,
   -1898.8079945653676,
   -1898.8079945653676,
   -1898.8079945653676,
   -1898.8188945651054,
   -1898.916994562745,
   -1898.916994562745,
   -1898.9278945624828,
   -1899.0259945601224,
   -1899.0259945601224,
   -1899.0368945598602,
   -1899.1349945574998,
   -1899.1349945574998,
   -1899.1458945572376,
   -1899.2439945548772,
   -1899.2439945548772,
   -1899.254894554615,
   -1899.3529945522546,
   -1899.3529945522546,
   -1899.3638945519924,
   -1899.461994549632,
   -1899.4728945493698,
   -1899.5709945470094,
   -1899.5709945470094,
   -1899.5818945467472,
   -1899.6799945443868,
   -1899.6908945441246,
   -1899.7889945417642,
   -1899.7889945417642,
   -1899.799894541502,
   -1899.8979945391416,
   -1899.9088945388794,
   -1900.0178945362568,
   -1900.1159945338964,
   -1900.1268945336342,
   -1900.2249945312738,
   -1900.2358945310116

In [21]:
np.mean(np.hstack(l))

-1905.7577733220617

In [16]:
l[0]

[[-1898.6989945679902,
  -1898.6989945679902,
  -1898.6989945679902,
  -1898.709894567728,
  -1898.8079945653676,
  -1898.8079945653676,
  -1898.8079945653676,
  -1898.8079945653676,
  -1898.8079945653676,
  -1898.8188945651054,
  -1898.916994562745,
  -1898.916994562745,
  -1898.9278945624828,
  -1899.0259945601224,
  -1899.0259945601224,
  -1899.0368945598602,
  -1899.1349945574998,
  -1899.1349945574998,
  -1899.1458945572376,
  -1899.2439945548772,
  -1899.2439945548772,
  -1899.254894554615,
  -1899.3529945522546,
  -1899.3529945522546,
  -1899.3638945519924,
  -1899.461994549632,
  -1899.4728945493698,
  -1899.5709945470094,
  -1899.5709945470094,
  -1899.5818945467472,
  -1899.6799945443868,
  -1899.6908945441246,
  -1899.7889945417642,
  -1899.7889945417642,
  -1899.799894541502,
  -1899.8979945391416,
  -1899.9088945388794,
  -1900.0178945362568,
  -1900.1159945338964,
  -1900.1268945336342,
  -1900.2249945312738,
  -1900.2358945310116,
  -1900.3339945286511,
  -1900.344894528

In [16]:
import itertools

In [20]:
a = (list(itertools.chain.from_iterable(x_boundary)))
b = (list(itertools.chain.from_iterable(a)))
np.mean(b)

-7687.721534009542

In [19]:
b

[-7690.556640625,
 -7690.45854062736,
 -7690.447640627623,
 -7690.349540629983,
 -7690.338640630245,
 -7690.240540632605,
 -7690.240540632605,
 -7690.229640632868,
 -7690.131540635228,
 -7690.131540635228,
 -7690.12064063549,
 -7690.02254063785,
 -7690.02254063785,
 -7690.02254063785,
 -7690.011640638113,
 -7689.913540640473,
 -7689.913540640473,
 -7689.902640640736,
 -7689.804540643096,
 -7689.804540643096,
 -7689.793640643358,
 -7689.695540645718,
 -7689.684640645981,
 -7689.586540648341,
 -7689.586540648341,
 -7689.575640648603,
 -7689.477540650963,
 -7689.466640651226,
 -7689.357640653849,
 -7689.259540656209,
 -7689.248640656471,
 -7689.237740656734,
 -7689.139640659094,
 -7689.0306406617165,
 -7689.019740661979,
 -7688.921640664339,
 -7688.910740664602,
 -7688.812640666962,
 -7688.801740667224,
 -7688.703640669584,
 -7688.692740669847,
 -7688.594640672207,
 -7688.4856406748295,
 -7688.474740675092,
 -7688.474740675092,
 -7688.376640677452,
 -7688.365740677715,
 -7688.267640680075

In [13]:
x_boundary

[[[-7690.556640625,
   -7690.45854062736,
   -7690.447640627623,
   -7690.349540629983,
   -7690.338640630245,
   -7690.240540632605,
   -7690.240540632605,
   -7690.229640632868,
   -7690.131540635228,
   -7690.131540635228,
   -7690.12064063549,
   -7690.02254063785,
   -7690.02254063785,
   -7690.02254063785,
   -7690.011640638113,
   -7689.913540640473,
   -7689.913540640473,
   -7689.902640640736,
   -7689.804540643096,
   -7689.804540643096,
   -7689.793640643358,
   -7689.695540645718,
   -7689.684640645981,
   -7689.586540648341,
   -7689.586540648341,
   -7689.575640648603,
   -7689.477540650963,
   -7689.466640651226,
   -7689.357640653849,
   -7689.259540656209,
   -7689.248640656471,
   -7689.237740656734,
   -7689.139640659094,
   -7689.0306406617165,
   -7689.019740661979,
   -7688.921640664339,
   -7688.910740664602,
   -7688.812640666962,
   -7688.801740667224,
   -7688.703640669584,
   -7688.692740669847,
   -7688.594640672207,
   -7688.4856406748295,
   -7688.47474067

In [23]:
x_boundary[0][0][0]

-7690.556640625

In [ ]:
sample_list = ['mouse1_sample1','mouse1_sample2','mouse1_sample3','mouse1_sample4',
              'mouse2_sample1','mouse2_sample2','mouse2_sample3','mouse2_sample4',


              ]
cell_id_list_all = []
x_list_all = []
y_list_all = []
slide_id_list_all = []
for sample in sample_list:
    segment_matrix_raw = pd.read_csv(f'{raw_path}/segmented_cells_{sample}.csv',dtype='str',delimiter=',',skiprows=1,names=['cell_id','z','x','y','slide_id'],quotechar='\"')
    cell_id_list = np.array(segment_matrix_raw['cell_id'])
    x_list = []
    y_list = []
    for i in range((segment_matrix_raw.shape[0])):
#         x_boundary = eval(segment_matrix_raw['x'][i])
#         a = (list(itertools.chain.from_iterable(x_boundary)))
#         b = (list(itertools.chain.from_iterable(a)))
#         cur_x = np.mean(b)
        
        
#         y_boundary = eval(segment_matrix_raw['y'][i])
#         a = (list(itertools.chain.from_iterable(y_boundary)))
#         b = (list(itertools.chain.from_iterable(a)))
#         cur_y = np.mean(b)


        x_boundary = eval(segment_matrix_raw['x'][i])
        y_boundary = eval(segment_matrix_raw['y'][i])
        cur_x = x_boundary[0][0][0]
        cur_y = y_boundary[0][0][0]
        print(segment_matrix_raw['cell_id'][i],cur_x,cur_y)
        
        # x_boundary = np.array(segment_matrix_raw['x'][i].split(', ')).astype('float')
        # y_boundary = np.array(segment_matrix_raw['y'][i].split(', ')).astype('float')
        # cur_x = np.mean(x_boundary)
        # cur_y = np.mean(y_boundary)
        x_list.append(cur_x)
        y_list.append(cur_y)
    slide_id_list = np.array(segment_matrix_raw['slide_id'])
    cell_id_list_all.extend(cell_id_list)
    x_list_all.extend(x_list)
    y_list_all.extend(y_list)
    slide_id_list_all.extend(slide_id_list)


    


In [10]:
adata.shape


(280327, 254)

In [13]:
len(cell_id_list_all)

401052

In [15]:
cell_id_list_filter = []
x_list_filter = []
y_list_filter = []
slide_id_list_filter = []
for i in range(len(cell_id_list_all)):
    if cell_id_list_all[i] in adata.obs_names:
        cell_id_list_filter.append(cell_id_list_all[i])
        x_list_filter.append(x_list_all[i])
        y_list_filter.append(y_list_all[i])
        slide_id_list_filter.append(slide_id_list_all[i])
        

In [17]:
adata_new = adata[cell_id_list_filter]

In [18]:
adata_new.obsm['spatial'] = np.vstack([x_list_filter,y_list_filter]).transpose()
adata_new.obs['slide_id'] = slide_id_list_filter

In [20]:
adata_new.write_h5ad(f'{h5ad_path}/all.h5ad')

In [30]:
adata_new = sc.read_h5ad(f'{h5ad_path}/all.h5ad')

In [32]:
for sample in np.unique(adata_new.obs['slide_id']):
    a_sample = adata_new[adata_new.obs['slide_id']==sample]
    # sc.pl.embedding(a_sample,basis='spatial',color='leiden')
    a_sample.write_h5ad(f'{h5ad_path}/{sample}.h5ad')